# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.builders import model_builder
from google.protobuf import text_format
import tensorflow as tf
from object_detection.protos import model_pb2
import time
from IPython.display import clear_output
from keras.models import load_model
from math import floor, ceil
import cv2

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')


## Env setup

In [ ]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [ ]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [ ]:
# What model to download.
MODEL_NAME = '../prod_faster_rcnn_resnet50_coco_res_512x384_iou_genration'
# MODEL_NAME = 'faster_rcnn_resnet50_coco_2018_01_28'
# MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28'

MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
test_image_path = "/home/wc-gpu/MasterThesis/session_data/122923_testing"
# test_image_path = "/Users/mbaltac/Home/MasterThesis/MasterThesis/labeled_images_shopping_sessions/labeled/122923_testing"
TEST_IMAGE_PATHS = sorted([os.path.join(test_image_path, f) for f in os.listdir(test_image_path) if f.endswith("jpg")])


# TEST_IMAGE_PATHS = [TEST_IMAGE_PATHS[0], TEST_IMAGE_PATHS[0]]


# Size, in inches, of the output images.
IMAGE_SIZE = (18, 12)

SESSIONS_PATH = "/home/wc-gpu/storage4tb/session_data_thesis/sessions160000_165000"

## Download Model

In [ ]:
# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#   file_name = os.path.basename(file.name)
#   if 'frozen_inference_graph.pb' in file_name:
#     tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [ ]:
# with tf.Graph().as_default():
#   od_graph_def = tf.GraphDef()
#   with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
#     serialized_graph = fid.read()
#     od_graph_def.ParseFromString(serialized_graph)
#     tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [ ]:
# category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
category_index = {1: {'id': 1, 'name': 'product'},
 2: {'id': 2, 'name': 'add'},
 3: {'id': 3, 'name': 'remove'},
 4: {'id': 4, 'name': 'move_from'},
 5: {'id': 5, 'name': 'move_to'}}

## Helper code

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [ ]:
def max_pool_dynamic_input(x, H_out, W_out):
    out = None
    (N, H, W, C) = x.shape
    pool_height = np.zeros((H_out), dtype="int32")
    pool_width = np.zeros((W_out), dtype="int32")
    
    div_h = int(H/H_out)
    mod_h = H%H_out
    div_w = int(W/W_out)
    mod_w = W%W_out
    pool_height = pool_height + div_h
    pool_width = pool_width + div_w
    pool_height[:mod_h] += 1
    pool_width[:mod_w] += 1
    
    out = np.zeros((N, H_out, W_out, C))
    for n in range(N):
        for h in range(H_out):
            for w in range(W_out):
                h1 = sum(pool_height[:h])
                h2 = h1 + pool_height[h]
                w1 = sum(pool_width[:w])
                w2 = w1 + pool_width[w]
                window = x[n, h1:h2, w1:w2, :]
                m = np.max(window, axis=0)
                m = np.max(m, axis=0)
                out[n,h,w,:] = m
    
    return out


def get_max_and_concat(*args):
    size = 0
    for arg in args:
        size += arg.shape[-1]
    final = np.zeros(size)
    
    arg_size = 0
    
    i = 0
    for arg in args:
        max_pool = max_pool_dynamic_input(np.expand_dims(arg, 0), 1, 1)[0,0,0,:]
        final[i: arg.shape[-1]+i] = max_pool
        i+= arg.shape[-1]  
    return final

def get_max_and_concat2(*args):
    size = 0
    for arg in args:
        size += arg.shape[-1]
    final = np.zeros(size)
    
    arg_size = 0

    for arg in args:
        for i in range(arg.shape[-1]):
            final[i+arg_size] = np.max(arg[:,:,i])
        arg_size += arg.shape[-1]
    
#     mn = np.mean(final)
#     std = np.std(final)
#     final = (final - mn)/std

    return final

def max_pooled(previous_features, features, label):
    box_prev = previous_features[
        floor(label[0]*previous_features.shape[0]):
        ceil(label[2]*previous_features.shape[0]), 
        floor(label[1]*previous_features.shape[1]):
        ceil(label[3]*previous_features.shape[1]),:]
    box_cur = features[
        floor(label[0]*features.shape[0]):
        ceil(label[2]*features.shape[0]), 
        floor(label[1]*features.shape[1]):
        ceil(label[3]*features.shape[1]),:]

    return get_max_and_concat(box_prev, previous_features, box_cur, features)

def roi_pool(features, label):
    box_features = features[
        floor(label[0]*features.shape[0]):
        ceil(label[2]*features.shape[0]), 
        floor(label[1]*features.shape[1]):
        ceil(label[3]*features.shape[1]),:]
    
    return max_pool_dynamic_input(np.expand_dims(box_features, 0), 1, 1)[0,0,0,:]


In [ ]:
a=np.random.rand(12, 13, 1024)

In [ ]:
b=np.random.rand(3, 4, 1024)

In [ ]:
c = get_max_and_concat(a, b)

In [ ]:
c1 = get_max_and_concat2(a,b)

In [ ]:
sum(c==c1) - len(c)

In [ ]:
c1

In [ ]:
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    """

    b1y1, b1x1, b1y2, b1x2 = bb1
    b2y1, b2x1, b2y2, b2x2 = bb2

#     b1y1, b1x1, b1y2, b1x2 = b1y1, b1x1, b1y1+b1y2, b1x1+b1x2
#     b2y1, b2x1, b2y2, b2x2 = b2y1, b2x1, b2y1+b2y2, b2x1+b2x2

    b1y1, b1x1, b1y2, b1x2 = b1x1, b1y1, b1x2, b1y2
    b2y1, b2x1, b2y2, b2x2 = b2x1, b2y1, b2x2, b2y2
    
    # determine the coordinates of the intersection rectangle
    x_left = max(b1x1, b2x1)
    y_top = max(b1y1, b2y1)
    x_right = min(b1x2, b2x2)
    y_bottom = min(b1y2, b2y2)

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (b1x2 - b1x1) * (b1y2 - b1y1)
    bb2_area = (b2x2 - b2x1) * (b2y2 - b2y1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou > 0.0
    assert iou < 1.0
    return iou


def filter_detections(out_boxes, out_scores, iou_tresh = .5, score_tresh = 0.3):
    fboxes, fscores, fclasses = [], [], []
    removed = set()
    
    filter_score = np.where(out_scores > score_tresh)
    print (filter_score)
    
    for i, b in reversed(list(enumerate(out_boxes))):
        if i not in removed:
            max_score = out_scores[i]
            max_idx = i
            for j in range(i+1, len(out_boxes)):
                if j not in removed and get_iou(b, out_boxes[j]) > .65:
                    if out_scores[j] > max_score:
                        removed.add(max_idx)
                        max_idx = j
                        max_score = out_scores[j]
                    else:
                        removed.add(j)

    for i in range(len(out_boxes)):
        if i not in removed:
            fboxes.append(out_boxes[i])
            fscores.append(out_scores[i])
            fclasses.append(out_classes[i])

    return fboxes, fscores, fclasses

In [ ]:
tf_config = tf.ConfigProto()
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.8

In [ ]:
def run_inference_for_single_image(image):
    with tf.Session(config=tf_config) as sess:

      # Get handles to input and output tensors
        ops = tf.get_default_graph().get_operations()
        all_tensor_names = {output.name for op in ops for output in op.outputs}
        tensor_dict = {}
        for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
        ]:
            tensor_name = key + ':0'
            if tensor_name in all_tensor_names:
                tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                  tensor_name)
        features = tf.get_default_graph().get_tensor_by_name(
            'FirstStageFeatureExtractor/resnet_v1_50/resnet_v1_50/block3/unit_6/bottleneck_v1/Relu:0')

        tensor_dict['features'] = tf.expand_dims(features, 0)
        
        image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

        # Run inference
        output_dict = sess.run(tensor_dict, 
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

        # all outputs are float32 numpy arrays, so convert types as appropriate
        output_dict['num_detections'] = int(output_dict['num_detections'][0])
        output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
        output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
        output_dict['detection_scores'] = output_dict['detection_scores'][0]
        output_dict['features'] = output_dict['features'][0][0]

        return output_dict

In [ ]:
def batch_gen(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i: min(i + n, len(l))]
        
def run_inference_for_batch(images, batch = 16):
    with tf.Session(config=tf_config) as sess:
#         batches = batch_gen(images, batch)
      # Get handles to input and output tensors
        ops = tf.get_default_graph().get_operations()
        all_tensor_names = {output.name for op in ops for output in op.outputs}
        tensor_dict = {}
        for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
        ]:
            tensor_name = key + ':0'
            if tensor_name in all_tensor_names:
                tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                  tensor_name)
        features = tf.get_default_graph().get_tensor_by_name(
            'FirstStageFeatureExtractor/resnet_v1_50/resnet_v1_50/block3/unit_6/bottleneck_v1/Relu:0')

        tensor_dict['features'] = tf.expand_dims(features, 0)
        
        image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
        
        # Run inference
        output_dict = sess.run(tensor_dict, 
                             feed_dict={image_tensor: images})

        # all outputs are float32 numpy arrays, so convert types as appropriate
        output_dict['num_detections'] = output_dict['num_detections'].astype(np.uint8)
        output_dict['detection_classes'] = output_dict[
          'detection_classes'].astype(np.uint8)
        output_dict['detection_boxes'] = output_dict['detection_boxes']
        output_dict['detection_scores'] = output_dict['detection_scores']
        output_dict['features'] = output_dict['features'][0]

        return output_dict

In [ ]:
#     ker_model = load_model(
#         "/home/wc-gpu/MasterThesis/models/research/object_detection/box_classification_checkpoint/keep/new_trained_one_class_acc0.86.hdf5")

session_path = "/home/wc-gpu/storage4tb/session_data_thesis/sessions160000_165000/123034//"
# session_path = "/home/wc-gpu/MasterThesis/session_data/122923_testing"
# 122873

image_paths = sorted([f for f in os.listdir(session_path) if f.endswith(".jpg")])
total_images = len(image_paths)
prev_features = None

images = []
for image_fname in image_paths:
    has_processed = False
    t0 = time.time()
    image_path = os.path.join(session_path, image_fname)

    image = Image.open(image_path)
    image_np = load_image_into_numpy_array(image)
    images.append(image_np)

In [ ]:
progress = 0
batch_size = 16
processed = 0
time_avg = 0

outputs = []

total_time = 0
box_predictions = []
tf.reset_default_graph()
with tf.Graph().as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
    
    image_batch = batch_gen(images, batch_size)        
    for img_b in image_batch:
        output_dict = run_inference_for_batch(img_b)
        outputs.append(output_dict)
        
        progress += batch_size
        processed += batch_size
        t1 = time.time()

        total = t1-t0
        
        time_avg = (total_time + total) / processed
        
        total_time += total
        
        log_text = " ".join(["progress ", ":", str(progress / total_images), 
                             str(progress), str(total_images)])
        log_text += " | "

        log_text += "".join(["process time ", str(total)])
        log_text += " | "
        log_text += "".join([" time passed: ", str(time_avg*processed) ," s, remaining: ", str(time_avg* (total_images - progress) ), " s"])
            

        print (log_text, end="                                       \r")

In [ ]:
def cos(vA, vB):
    return np.dot(vA, vB) / (np.sqrt(np.dot(vA,vA)) * np.sqrt(np.dot(vB,vB)))
def normalize(x):
    mn = np.mean(x, axis=0)
    std = np.std(x, axis=0)
    x = (x - mn)/std
    return mn, std

In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Model

In [ ]:
mobile_model = MobileNetV2(input_shape=(96, 96, 3))

In [ ]:
mobile_model.get_layer("global_average_pooling2d_1")

feature_extractor = Model(mobile_model.input, mobile_model.get_layer("global_average_pooling2d_1").output)

In [ ]:
compare = load_model("/home/wc-gpu/MasterThesis/models/research/object_detection/od_api_tf_my_notebooks/checkpoint_similar/2019-04-04weights-epoch30-val_acc0.83-val_loss0.74.hdf5")

In [ ]:
new_outs = []
prev_out = None
prev_image = None
i=0
for big_dict in outputs:
    for j in range(len(big_dict["num_detections"])):
        output_dict = {}
        for k in big_dict.keys():
            output_dict[k] = big_dict[k][j]    
        new_det = []
        new_cls = []
        image_np = images[i*batch_size + j]
        if prev_out:
            prev_det = prev_out['detection_boxes']
            
            nfound = set()
            ofound = set()
            for ni in range(len(output_dict['detection_boxes'])):
                if output_dict['detection_scores'][ni] < .5:
                    continue
                nb = output_dict['detection_boxes'][ni]
                int_box = [int(nb[0]*image_np.shape[0]),
                int(nb[1]*image_np.shape[1]),
                int(nb[2]*image_np.shape[0]),
                int(nb[3]*image_np.shape[1])]
                
                ncrop = image_np[int_box[0]:int_box[2] , int_box[1]:int_box[3], :]
                ncrop = cv2.resize(ncrop, (96, 96))/255.
#                 cv2.imshow("A", ncrop)
                found = False
                for oi in range(len(prev_det)):
                    if prev_out['detection_scores'][oi] < .6:
                        continue
                    ob = prev_det[oi]
                    int_box = [int(ob[0]*image_np.shape[0]),
                    int(ob[1]*image_np.shape[1]),
                    int(ob[2]*image_np.shape[0]),
                    int(ob[3]*image_np.shape[1])]
                    ocrop = prev_image[int_box[0]:int_box[2] , int_box[1]:int_box[3], :]
                    
                    ocrop = cv2.resize(ocrop, (96, 96))/255.
                    
                    similarity = compare.predict([np.expand_dims(ocrop, 0), np.expand_dims(ncrop, 0)])[0]
                    print(similarity)
                    if 1-similarity < 0.5:
                        found = True
                        break
                    
                if not found:
                    print ("found", )
                    new_det.append(nb)
                    new_cls.append(2)

#             for ob in prev_det:
#                 found = False
#                 for nb in output_dict['detection_boxes']:
#                     if get_iou(ob, nb) > 0.8:
#                         found = True
#                         break
#                 if not found:
#                     new_det.append(ob)
#                     new_cls.append(3)      

#         else:
#             new_cls = len(output_dict['detection_boxes'])*[2]
#             new_det = output_dict['detection_boxes']
        
        new_dict = dict()
        new_dict.update(output_dict)
        
        
        new_dict['detection_boxes'] = np.array(new_det)
        new_dict['detection_classes'] = np.array(new_cls)
        new_dict['detection_scores'] = np.ones(len(new_det))
        del new_dict["features"]
        
        new_outs.append(new_dict)
        
        prev_out = output_dict
        prev_image = image_np
    i+=1

In [ ]:
from IPython.display import display, clear_output
clear_output(wait=True)

i=0
for i in range(len(new_outs)):
    output_dict = new_outs[i]
    not_same = np.where(output_dict["detection_classes"] > 0)
    image_np = images[i]
    image_np_out = np.copy(image_np)
    vis_util.visualize_boxes_and_labels_on_image_array(
          image_np_out,
          output_dict['detection_boxes'][not_same],
          output_dict["detection_classes"][not_same],
          output_dict['detection_scores'][not_same],
          category_index,
          use_normalized_coordinates=True,
          line_thickness=5,
          min_score_thresh=.5)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np_out)


In [ ]:
# from IPython.display import display, clear_output
# clear_output(wait=True)

# i=0
# for big_dict in outputs:
#     for j in range(len(big_dict["num_detections"])):
#         output_dict = {}
#         for k in big_dict.keys():
#             output_dict[k] = big_dict[k][j]
#         not_same = np.where(output_dict["detection_classes"] > 0)
#         image_np = images[i*batch_size + j]
#         image_np_out = np.copy(image_np)
#         vis_util.visualize_boxes_and_labels_on_image_array(
#               image_np_out,
#               output_dict['detection_boxes'][not_same],
#               output_dict["detection_classes"][not_same],
#               output_dict['detection_scores'][not_same],
#               category_index,
#               use_normalized_coordinates=True,
#               line_thickness=5,
#               min_score_thresh=.5)
#         plt.figure(figsize=IMAGE_SIZE)
#         plt.imshow(image_np_out)
#     i+=1


In [ ]:
new_outs = []
prev_out = None
prev_image = None
i=0
for big_dict in outputs:
    for j in range(len(big_dict["num_detections"])):
        output_dict = {}
        for k in big_dict.keys():
            output_dict[k] = big_dict[k][j]    
        new_det = []
        new_cls = []
        image_np = images[i*batch_size + j]
        if prev_out:
            prev_det = prev_out['detection_boxes']
            
            nfound = set()
            ofound = set()
            for ni in range(len(output_dict['detection_boxes'])):
                if output_dict['detection_scores'][ni] < .6:
                    continue
                nb = output_dict['detection_boxes'][ni]
                n_features = roi_pool(output_dict["features"], nb)
#                 normalize(n_features)
                found = False
                for oi in range(len(prev_det)):
                    if prev_out['detection_scores'][oi] < .6:
                        continue
                    ob = prev_det[oi]
                    o_features = roi_pool(prev_out["features"], ob)
#                     normalize(o_features)
                    print (cos(n_features, o_features))
                    if oi not in ofound and cos(n_features, o_features) >.5:
                        found = True
                        ofound.add(oi)
                        nfound.add(ni)
                        break
                if not found:
                    print ("forund", cos(n_features, o_features))
                    new_det.append(nb)
                    new_cls.append(2)

#             for ob in prev_det:
#                 found = False
#                 for nb in output_dict['detection_boxes']:
#                     if get_iou(ob, nb) > 0.8:
#                         found = True
#                         break
#                 if not found:
#                     new_det.append(ob)
#                     new_cls.append(3)      

        else:
            new_cls = len(output_dict['detection_boxes'])*[2]
            new_det = output_dict['detection_boxes']
        
        new_dict = dict()
        new_dict.update(output_dict)
        
        
        new_dict['detection_boxes'] = np.array(new_det)
        new_dict['detection_classes'] = np.array(new_cls)
        new_dict['detection_scores'] = np.ones(len(new_det))
        del new_dict["features"]
        
        new_outs.append(new_dict)
        
        prev_out = output_dict
    i+=1